In [10]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())  


True

In [11]:
def get_chat_completion(prompt, model) ->str: 
    response = llm.invoke(prompt) 
    return response.content 

llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model="llama-3.3-70b-versatile",
    temperature=0
)


In [21]:
template_string = """
You are an expert data analyst.
The user has a question about a plunger lift system, which has time series data for the following features:
1. Arrival Speed
2. Arrival Time Remaining
3. Casing Pressure
4. Current Non-Arrival Time
5. Down Hole Pressure
6. Line Pressure
7. Sales Meter Flow Rate
8. Sales Meter Static Pressure
9. Tubing Pressure

Each feature is stored in a separate CSV file (in directory ./La Vista 1H) with two columns: a 'timestamp' column and a column with the feature name.

Given the user's question, do the following:
1. Identify the relevant feature(s) from the list above. If none are found, respond with: **"Feature is not available."**
2. Identify any time constraints mentioned (e.g., specific date/time or time range).
3. Output a **Python Pandas code** that filters the CSV file(s) to get the requested information.

Output only the code, without any additional text or explanations.

User query: {query}
"""


In [22]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string) 

In [28]:
def extract_code_from_markdown(llm_output: str) -> str:
    if llm_output.startswith("```"):
        return "\n".join(
            line for line in llm_output.splitlines()
            if not line.strip().startswith("```")
        )
    return llm_output

user_query = "give me the latest 5 reading of arival speed"
message = prompt_template.format_messages(query=user_query)
resp = get_chat_completion(message, llm)
extracted_code = extract_code_from_markdown(resp)
extracted_code

"import pandas as pd\nimport os\n\n# Load Arrival Speed data\narrival_speed_df = pd.read_csv('./La Vista 1H/Arrival Speed.csv')\n\n# Sort by timestamp in descending order and get the latest 5 readings\nlatest_arrival_speed = arrival_speed_df.sort_values(by='timestamp', ascending=False).head(5)\n\nprint(latest_arrival_speed)"

In [29]:
exec(extracted_code)

                 timestamp    value
7200  2025-07-09T08:08:23Z  1057.80
7199  2025-07-09T08:06:23Z  1057.80
7198  2025-07-09T08:04:23Z  1057.80
7197  2025-07-09T08:02:23Z  1057.80
7196  2025-07-09T08:00:23Z  1122.95
